<a href="https://colab.research.google.com/github/r-diazp/deepL/blob/master/x_ray.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![texto alternativo](https://soyoungpark.github.io/assets/images/tensorflowtraffic/tf.png)

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import os

#Habilitar TensroBoard 
import datetime
%load_ext tensorboard
!rm -rf ./logs/ 
log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Configuración y conexion con Kaggle

In [2]:
#Configurar API Kaggle en Google Colab (#! cp kaggle.json ~/.kaggle/)
! pip install -q kaggle
! mkdir ~/.kaggle
! cp /content/drive/MyDrive/Kaggle_API/kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Dowload competition
! kaggle competitions download <name-of-competition>

#Dowload dataset
! kaggle datasets download <name-of-dataset>

In [3]:
#Descargar dataset 
! kaggle datasets download tolgadincer/labeled-chest-xray-images

100% 1.17G/1.17G [00:08<00:00, 176MB/s]
100% 1.17G/1.17G [00:08<00:00, 153MB/s]


In [ ]:
#Descomprimir archivo 
file_dataset = 'labeled-chest-xray-images'
! unzip {file_dataset}

# Cargar & normalizar tensores

In [5]:
#Directorios 
dataset_path_new = "./chest_xray/"
train_dir = os.path.join(dataset_path_new, "train")
validation_dir = os.path.join(dataset_path_new, "test")

#Parametros de carga 
imagen_size = 150
batch_size = 126
color_mode = "grayscale"
data_train = ImageDataGenerator(rescale=1/255.)
data_test = ImageDataGenerator(rescale=1/255.)

#CARGAR Y ESTANDARIZAR TAMAÑO DE LAS IMAGENES 
print("Imagenes de entrenamiento detectadas:")
image_train = data_train.flow_from_directory(
    train_dir, 
    target_size=(imagen_size,imagen_size),
    batch_size=batch_size, 
    color_mode=color_mode,
    class_mode="categorical")
    
print("\nImagenes de prueba detectadas:")
image_test = data_test.flow_from_directory(
    validation_dir,
    target_size=(imagen_size,imagen_size), 
    batch_size=batch_size,
    color_mode=color_mode,
    class_mode="categorical")

print("""\n- Nombre imagenes de Entrenamiento: imagenes_train
- Nombre imagenes de prueba: imagenes_test""") 

Imagenes de entrenamiento detectadas:
Found 5232 images belonging to 2 classes.

Imagenes de prueba detectadas:
Found 624 images belonging to 2 classes.

- Nombre imagenes de Entrenamiento: imagenes_train
- Nombre imagenes de prueba: imagenes_test


# Arquitectura de la CNN

In [ ]:
#Arquitectura de la Red Neuronal Convolucional (Ajustar imagen a 150x150 pixeles)
#Parametros
activation_func = 'relu'
loss = 'categorical_crossentropy'
learning_rate = 0.0001

modelo = tf.keras.models.Sequential([
    # El tamaño de la imagen de entrada debe ser de 150x150 pixeles con 3 bytes color (input_shape)
    # Primera capa de convolucion con 64 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2 (Función activación: Rectificador Lineal)
    tf.keras.layers.Conv2D(64, (3,3), activation = activation_func, 
                           input_shape=(imagen_size, imagen_size, 1)), 
    tf.keras.layers.MaxPooling2D(2, 2),
    
    # Segunda capa de convolución  con 64 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2 
    tf.keras.layers.Conv2D(64, (3,3), activation = activation_func),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Tercera capa de convolución  con 128 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation = activation_func),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Cuarta capa de convolución  con 128 filtros de tamaño 3x3 y capa MaxPooling tamaño 2x2
    tf.keras.layers.Conv2D(128, (3,3), activation = activation_func),
    tf.keras.layers.MaxPooling2D(2,2),
    
    # Capa de aplando para la entrada a la red neuronal y Dropout para evitar overfitting 
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),
    
    # 512 neuronas de capa oculta y funcion de salida softmax (Clasificará 3 tipos de imagenes)
    tf.keras.layers.Dense(512, activation = activation_func),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(512, activation = activation_func),
    tf.keras.layers.Dense(2, activation='softmax')
])

#Documentación del RMSprop: https://keras.io/api/optimizers/rmsprop/
modelo.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate = learning_rate),
               loss = loss, metrics=['accuracy'])
modelo.summary()

In [19]:
#Guardar el mejor modelo 
checkpoint_filepath = '/content/model/check'
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath = checkpoint_filepath,
    save_weights_only = True,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)
early_stopping = tf.keras.callbacks.EarlyStopping(patience=5)

In [20]:
#@title Entrenar a la Red Neuronal
epochs =  20#@param {type:"number"}

!rm -rf ./logs/ #Limpiar datos TensorBoard 
modelo.fit(image_train,
           validation_data = image_test, epochs = epochs,            
           callbacks=[model_checkpoint,  
                      tensorboard_callback])

Epoch 1/20
42/42 [==============================] - 53s 1s/step - loss: 0.5643 - accuracy: 0.7380 - val_loss: 0.5696 - val_accuracy: 0.6282
Epoch 2/20
42/42 [==============================] - 50s 1s/step - loss: 0.4418 - accuracy: 0.7882 - val_loss: 0.4241 - val_accuracy: 0.7692
Epoch 3/20
42/42 [==============================] - 49s 1s/step - loss: 0.3142 - accuracy: 0.8656 - val_loss: 0.3157 - val_accuracy: 0.8750
Epoch 4/20
42/42 [==============================] - 51s 1s/step - loss: 0.2460 - accuracy: 0.8966 - val_loss: 0.2723 - val_accuracy: 0.8942
Epoch 5/20
42/42 [==============================] - 49s 1s/step - loss: 0.1915 - accuracy: 0.9234 - val_loss: 0.2726 - val_accuracy: 0.9038
Epoch 6/20
42/42 [==============================] - 50s 1s/step - loss: 0.1645 - accuracy: 0.9358 - val_loss: 0.3438 - val_accuracy: 0.8766
Epoch 7/20
42/42 [==============================] - 50s 1s/step - loss: 0.1488 - accuracy: 0.9436 - val_loss: 0.3191 - val_accuracy: 0.8894
Epoch 8/20
42/42 [==

In [ ]:
%tensorboard --logdir logs

In [22]:
#Cargamos modelo con mejor resultados 
modelo.load_weights(checkpoint_filepath)

In [23]:
#Validacion datos
set_datos=image_test
valid_loss, exactitud_test = modelo.evaluate(set_datos)
print("""{} imagenes analizadas 
El modelo clasifico {:,.0f} imagenes correctamente, {:,.2f}% de exactitud""".format(len(set_datos.filenames),
                                                        len(set_datos.filenames)*exactitud_test,
                                                        exactitud_test*100,) )

5/5 [==============================] - 5s 1s/step - loss: 0.2726 - accuracy: 0.9038
624 imagenes analizadas 
El modelo clasifico 564 imagenes correctamente, 90.38% de exactitud


In [34]:
#@title Procesar imagen
# Identificar imagenes 
import numpy as np
from google.colab import files
from keras.preprocessing import image

#Cargar y Predecir imagenes 
uploaded = files.upload()
for fn in uploaded.keys():   
  path = fn
  img = image.load_img(path, target_size = (imagen_size,imagen_size),
                       color_mode = color_mode)
  x = image.img_to_array(img)
  x = np.expand_dims(x, axis=0)
  x= x/255
  #imagen = np.vstack([x])
  classes = modelo.predict(x,batch_size = imagen_size)
print('Archivo cargado... \nProcesando predicción:')
print(f'- Probabilidad normal: {np.round(classes[:,0].flatten()*100, 2)}%' )
print(f'- Probabilidad neumonia: {np.round(classes[:,1].flatten()*100, 2)}%' )

Saving NORMAL-1012843-0001.jpeg to NORMAL-1012843-0001.jpeg
Archivo cargado... 
Procesando predicción:
- Probabilidad normal: [98.82]%
- Probabilidad neumonia: [1.18]%
